<font size="5">Packages and Data</font>

In [63]:
#numcosmo packages
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import math
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

In [64]:
#math and stats packages
import numpy as np
from numpy import array
from numpy.lib.scimath import log10

import scipy as sp
from scipy.integrate import quad
from scipy.optimize import minimize

import pandas as pd

In [65]:
ser = Ncm.Serialize.new(0)
Ncm.cfg_init()
ser = Ncm.Serialize.new(0)
data = ser.from_file("/home/dougma13/NumCosmo/data/nc_data_snia_diag_legacy.obj")
length = data.y.len()

In [67]:
mu_o = []
sig_o = []
z_o = []

for i in range(length):
    mu_i = data.y.get(i)
    sig_i = data.sigma.get(i)
    z_i = data.x.get(i)
    
    mu_o.append(mu_i)
    sig_o.append(sig_i)
    z_o.append(z_i)

#speed of light
c = 3 * 10 ** 5 #km/s

<font size="6">a)</font> $\;$ Given that 

$$ \theta = (H_0, \Omega_{k0}, \Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda0}), $$

that our likelihood is equal to

$$L(D|\theta) = e^{-\frac{1}{2} \sum_i \frac{(\mu - \mu_i)^2}{\sigma_i^2}}, $$

and that $\Omega_{k0} = \Omega_{r0} = 0$, calculate the best fit, the fisher information and using the likelihood ratios test, calculate the confidence levels for $1\sigma$ , $2\sigma$ and $3\sigma$. Confidence levels need to be calculated only with two parameters at a time. 

<font size="4"> Likelihood Function (-2lnL)</font>

In [91]:
#individual factor of sum in -2lnL (L being the likelihood)
def chi2_i(z, theta, mu, sig):
    
    #inverse of the Hubble function
    def E_inv(z, theta):
        return (theta[0] + theta[1]*(1 + z)**3)**(-0.5)
    
    #comoving distance with no dimensions
    def D_c(z, theta):
        return quad(E_inv, 0, z, args = theta[0:2])[0]
    
    ##temporal comoving distance
    #D_t = D_c #for when omega_k = 0
    
    #luminostiy distance
    def D_L(z, theta):
        return (1 + z)*D_c(z, theta)
    
    #modular distance 
    def mu_c(z, theta):
        return 5*log10(D_L(z, theta)) + 25 + 5*log10(c/theta[2])
    
    return ((mu_c(z, theta) - mu)**2)/(sig**2) 

#actual -2lnL summing over all factors
def chi2(theta):
    chi2_list = []
    for i in range(length):
        chi2_list.append(chi2_i(z_o[i], theta, mu_o[i], sig_o[i]))
    return sum(chi2_list)

<font size="4"> Best Fit (Minimize -2lnL)

In [93]:
param = [0.70, 0.25, 70.0]


mini = minimize(chi2, x0 = param, method='Nelder-Mead', bounds = ((0,1), (0,1), (0, None)), tol=1e-6)

print(f'[ omega_l  omega_m  H0 ] \n {mini.x} ')

[ omega_l  omega_m  H0 ] 
 [ 0.71600744  0.25476161 71.11068382] 


<font size="6">b)</font> $\;$ Now repeat everything from a), except with all free parameters.

<font size="4"> Likelihood Function (-2lnL)</font>

In [94]:
#individual factor of sum in -2lnL (L being the likelihood)
def chi2_i(z, theta, mu, sig):
    
    #inverse of the Hubble function
    def E_inv(z, theta):
        return (theta[0] + theta[3]*(1 + z)**2 + theta[1]*(1 + z)**3 + theta[4]*(1 + z)**4)**(-0.5)
    
    #comoving distance with no dimensions
    def D_c(z, theta):
        return quad(E_inv, 0, z, args = theta)[0]
    
    #temporal comoving distance
    def D_t(z, theta):
        return np.sinh((theta[3])**0.5 * D_c(z, theta))/(theta[3])**0.5
    
    #luminostiy distance
    def D_L(z, theta):
        return (1 + z)*D_c(z, theta)
    
    #modular distance 
    def mu_c(z, theta):
        return 5*log10(D_L(z, theta)) + 25 + 5*log10(c/theta[2])
    
    return ((mu_c(z, theta) - mu)**2)/(sig**2) 

#actual -2lnL summing over all factors
def chi2(theta):
    chi2_list = []
    for i in range(length):
        chi2_list.append(chi2_i(z_o[i], theta, mu_o[i], sig_o[i]))
    return sum(chi2_list)

<font size="4"> Best Fit (Minimize -2lnL)

In [95]:
params = [0.70, 0.25, 70.0, 0.02, 1e-05]

mini = minimize(chi2, x0 = params, method='Nelder-Mead', bounds = ((0,1), (0,1), (0, None), (0,1), (0,1)), tol=1e-6)

print(f' omega_l = {mini.x[0]} \n omega_m = {mini.x[1]} \n H0 = {mini.x[2]} \n omega_k = {mini.x[3]} \n omega_r = {mini.x[4]}')

 omega_l = 0.9854003856093903 
 omega_m = 0.3506111915643809 
 H0 = 60.6171107483821 
 omega_k = 0.0 
 omega_r = 0.0


In [ ]:
70.0, 0.02, 0.25, 0.70, 1e-05]
H0, Omega_k0, Omega_m0, Omega_l0, Omega_r0 